In [1]:
import pandas as pd
from sqlalchemy import create_engine, text
import pymysql
import re
import time
from datetime import datetime
import os
import logging
import sshtunnel
from sshtunnel import SSHTunnelForwarder
import paramiko


user_name='trishala/'

ssh_host = '10.120.1.50'
ssh_username = 'research-user'
localhost = '127.0.0.1'

# prod
ezcapc_user = 'read_only'
ezcapc_pwd = 'WHC63fKyjudfSz2U'
ezcapc_host = 'ezcapc-webdb-production-readrep.cncmk5ndlbjo.us-east-1.rds.amazonaws.com'
ezcapc_db_name = 'CAPC_APIGATEWAY'
env='production'
paas_user = 'read_only'
paas_pwd = 'eUN2cORh1sCRG998'
paas_host = 'paas-production-readrep.cncmk5ndlbjo.us-east-1.rds.amazonaws.com'
paas_db_name = 'FACILITY_SERVICE'

# # stag
# ezcapc_user = 'read_only'
# ezcapc_pwd = '3j4A3PaQc8BV050R'
# ezcapc_host = 'ezcapc-staging.cncmk5ndlbjo.us-east-1.rds.amazonaws.com'
# ezcapc_db_name = 'CAPC_APIGATEWAY'
# env='staging'
# paas_user = 'read_only'
# paas_pwd = 's9a843QvgkPje5SD'
# paas_host = 'paas-staging.cncmk5ndlbjo.us-east-1.rds.amazonaws.com'
# paas_db_name = 'METADATA_SERVICE'



# ezcapc_db_connection_str = 'mysql+pymysql://{}:{}@{}/{}'.format(ezcapc_user,ezcapc_pwd,ezcapc_host,ezcapc_db_name)
# ezcapc_db_connection = create_engine(ezcapc_db_connection_str)

# paas_db_connection_str = 'mysql+pymysql://{}:{}@{}'.format(paas_user,paas_pwd,paas_host)
# paas_db_connection = create_engine(paas_db_connection_str)

In [2]:
def open_ssh_tunnel(verbose=False):
    if verbose:
        sshtunnel.DEFAULT_LOGLEVEL = logging.DEBUG
    
    global ezcapc_tunnel
    global paas_tunnel
    ezcapc_tunnel = SSHTunnelForwarder(
        (ssh_host, 22),
        ssh_username = ssh_username,
        ssh_pkey='/home/trishalaswain/pem/research_user.pem',
        remote_bind_address = (ezcapc_host, 3306)
    )
    paas_tunnel = SSHTunnelForwarder(
        (ssh_host, 22),
        ssh_username = ssh_username,
        ssh_pkey='/home/trishalaswain/pem/research_user.pem',
        remote_bind_address = (paas_host, 3306)
    )
    ezcapc_tunnel.start()
    paas_tunnel.start()
    print(ezcapc_tunnel.is_active)
    print(paas_tunnel.is_active)

In [3]:
def mysql_connect():
    global ezcapc_connection
    global paas_connection
    ezcapc_connection = pymysql.connect(
        host=localhost,
        user=ezcapc_user,
        passwd=ezcapc_pwd,
        database=ezcapc_db_name,
        port=ezcapc_tunnel.local_bind_port
    )

    paas_connection = pymysql.connect(
        host=localhost,
        user=paas_user,
        passwd=paas_pwd,
        database=paas_db_name,
        port=paas_tunnel.local_bind_port
    )

In [4]:
open_ssh_tunnel()
mysql_connect()

True
True


In [5]:
# import os
# import pandas as pd

# folder = "/home/trishalaswain/ClientData/SCP/01_to_16_tsv/SCP/"
# dfs = []
# for filename in os.listdir(folder):
#     if (filename.startswith('SCP-1388-') or
#         filename.startswith('SCP-21-') or
#         filename.startswith('SCP-2464-') or
#         filename.startswith('SCP-2497-') or
#         filename.startswith('SCP-2586-') or
#         filename.startswith('SCP-3228-') or
#         filename.startswith('SCP-36-') or
#         filename.startswith('SCP-45-') or
#         filename.startswith('SCP-5645-') or
#         filename.startswith('SCP-7198-')) and filename.endswith("31.tsv"):
#         print(filename)
#         print(os.path.join(folder, filename))
#         dfs.append(pd.read_table(os.path.join(folder, filename),index_col=False))
# df = pd.concat(dfs)

In [6]:
acc=[]
for i in open('/home/trishalaswain/ClientData/Other/Accounts.txt','r').readlines():
    acc.append(i.rstrip('\n'))

In [7]:
len(acc)

40

In [45]:
df_all=[]

# query='''SELECT DISTINCT
#     am.account_number,dpd.ocr_request_id, ec.code AS final_code, dc.code as suggested_code
# FROM
#     account_mst am
#         JOIN
#     document_mst dm ON dm.account_id = am.id
#         JOIN
#     document_processing_detail dpd ON dpd.document_id = dm.id
#         INNER JOIN
#     encounter_mst em ON am.id = em.account_id
#         INNER JOIN
#     document_code dc ON dc.document_id = dm.id
#         INNER JOIN
#     eandm_code ec ON ec.encounter_id = em.id
# WHERE
#     dpd.ocr_request_id IN ({})
#     AND dc.code_category='EANDM';'''.format(','.join(['"'+str(x)+'"' for x in acc]))

# 
# for f_id in type_id:
query='''
SELECT DISTINCT
    dpd.ocr_request_id,
    am.account_number,
    dpd.nlp_request_s3_path,
    dpd.nlp_ner_entities_result_s3_path,
    am.facility_id,
    dm.service_date,
    dpd.nlp_pcs_xml_result_s3_path,
    dpd.preprocessing_request_s3_path,
    dpd.preprocessing_result_s3_path,
    dm.orignal_path,
    dm.type_id,
    dpd.id as dpd_id,
    dpd.created_date,
    dpd.ocr_request_s3_path,
    dpd.ocr_result_s3_path,
    dpd.eandmcs_request_s3_path,
    dpd.eandmcs_result_s3_path,
    dpd.ruleengine_request_s3_path,
    dpd.cptcs_request_s3_path,
    dpd.cptcs_result_s3_path,
    dm.account_id,
    em.id as encounter_id,
    dc.code AS suggested_code,
    dpd.id AS DPD_ID,
    dm.path as filePath,
    dm.id AS document_id
FROM
    account_mst am
        JOIN
    document_mst dm ON dm.account_id = am.id
    AND 
    dm.is_active = 1
        JOIN
    document_processing_detail dpd ON dpd.document_id = dm.id
        INNER JOIN
    encounter_mst em ON am.id = em.account_id and em.physician_id=dm.physician_id and em.service_date=dm.service_date
        INNER JOIN
    document_code dc ON dc.document_id = dm.id
WHERE
        is_physician_document=1
        and am.is_active=1
        and am.account_number in ({})
        and dpd.id = (SELECT MAX(id)
        FROM document_processing_detail
        WHERE document_id = dm.id)
    order by dm.created_date desc;
'''.format(','.join(['"'+str(x)+'"' for x in acc]))


    #     INNER JOIN
    # eandm_code ec ON ec.encounter_id = em.id
# and dc.code_category = 'EANDM'
# dc.code_category = 'EANDM'
# and am.client_id = 15

In [46]:
# query = '''SELECT DISTINCT
#     am.account_number,
#     dpd.nlp_request_s3_path,
#     dpd.nlp_ner_entities_result_s3_path,
#     am.facility_id,
#     dm.service_date,
#     dpd.nlp_pcs_xml_result_s3_path,
#     dpd.preprocessing_request_s3_path,
#     dpd.preprocessing_result_s3_path,
#     dm.orignal_path,
#     dm.type_id,
#     dpd.created_date,
#     dpd.ocr_request_s3_path,
#     dpd.ocr_result_s3_path,
#     dpd.eandmcs_request_s3_path,
#     dpd.eandmcs_result_s3_path,
#     dpd.ruleengine_request_s3_path,
#     dpd.cptcs_request_s3_path,
#     dpd.cptcs_result_s3_path,
#     dm.account_id,
#     em.id as encounter_id,
#     dc.code AS suggested_code,
#     dpd.id AS DPD_ID,
#     dm.path as filePath,
# --     ec.code AS final_code,
#     dm.id AS document_id
# FROM
#     account_mst am
#         JOIN
#     document_mst dm ON dm.account_id = am.id
#     AND 
#     dm.is_active = 1
#         JOIN
#     document_processing_detail dpd ON dpd.document_id = dm.id
#         INNER JOIN
#     encounter_mst em ON am.id = em.account_id and em.physician_id=dm.physician_id and em.service_date=dm.service_date
#         INNER JOIN
#     document_code dc ON dc.document_id = dm.id
# --         INNER JOIN
# --     eandm_code ec ON ec.encounter_id = em.id
# WHERE
#         is_physician_document=1
#         and am.is_active=1
#         and facility_id = 168
#         and dc.code_category = 'EANDM'
#         and dpd.id = (SELECT MAX(id)
#         FROM document_processing_detail
#         WHERE document_id = dm.id)
#         and dpd.created_date >= '2023-08-01'
#     order by dm.created_date desc;'''

In [47]:
# '89541866' in acc

In [48]:
df_query = pd.read_sql(query, con=ezcapc_connection)

In [49]:
print(df_query.to_markdown())

|     | ocr_request_id   |   account_number | nlp_request_s3_path                                                                                        | nlp_ner_entities_result_s3_path                                                                       |   facility_id | service_date        | nlp_pcs_xml_result_s3_path                                                                               | preprocessing_request_s3_path                                                                               | preprocessing_result_s3_path                                                                               | orignal_path   |   type_id |   dpd_id | created_date        | ocr_request_s3_path   | ocr_result_s3_path   | eandmcs_request_s3_path                                                                                | eandmcs_result_s3_path                                                                                | ruleengine_request_s3_path                                 

In [50]:
# df_query.service_date = pd.to_datetime(df_query.service_date)
# df_query = df_query[df_query.service_date >= '2023-07-']

In [51]:
# df_query.to_excel('/home/trishalaswain/ClientData/SCP/BYRD_OCR_TEST/mappings.xlsx')

In [52]:
# set(acc)-set(df_query.encounter_id.to_list())

In [53]:
df_query.account_number.value_counts()

550001767935    33
550001755325    32
550001763465    31
550001774527    30
550001767934    30
550001774480    29
550001767681    29
550001754826    28
550001754790    28
550001754791    26
550001767547    25
550001757196    24
550001757541    23
550001754846    23
550001771465    23
550001771886    20
550001748269    19
550001767751    17
550001753113    17
550001769462    16
550001753280    15
550001769625    14
550001753141    14
550001771905    14
550001747743    14
550001753142    14
550001747854    14
550001767167    13
550001753284    13
550001757236    13
550001747411    12
550001769640    12
550001765943    11
550001753144    11
550001769502    10
550001777290    10
550001747381    10
550001762968     9
550001753228     7
550001767620     1
Name: account_number, dtype: int64

In [54]:
df_query

,ocr_request_id,account_number,nlp_request_s3_path,nlp_ner_entities_result_s3_path,facility_id,service_date,nlp_pcs_xml_result_s3_path,preprocessing_request_s3_path,preprocessing_result_s3_path,orignal_path,...,eandmcs_result_s3_path,ruleengine_request_s3_path,cptcs_request_s3_path,cptcs_result_s3_path,account_id,encounter_id,suggested_code,DPD_ID,filePath,document_id
0,None,550001774480,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/nlp/cdp-xml/...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/nlp/ner/2023...,430,2023-08-21 04:00:00,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/nlp/pcs-xml/...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/preprocessin...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/preprocessin...,None,...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/eandmcs/2023...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/ruleengine/2...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/cptcs/202308...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/cptcs/202308...,10943184,9270967,E11.9,12096201,SJRMC/data/CM_ADLT_OPHTHLM/document/20230821/3...,55907319
1,None,550001774480,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/nlp/cdp-xml/...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/nlp/ner/2023...,430,2023-08-21 04:00:00,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/nlp/pcs-xml/...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/preprocessin...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/preprocessin...,None,...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/eandmcs/2023...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/ruleengine/2...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/cptcs/202308...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/cptcs/202308...,10943184,9270967,M32.9,12096201,SJRMC/data/CM_ADLT_OPHTHLM/document/20230821/3...,55907319
2,None,550001774480,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/nlp/cdp-xml/...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/nlp/ner/2023...,430,2023-08-21 04:00:00,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/nlp/pcs-xml/...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/preprocessin...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/preprocessin...,None,...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/eandmcs/2023...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/ruleengine/2...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/cptcs/202308...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/cptcs/202308...,10943184,9270967,90853,12096201,SJRMC/data/CM_ADLT_OPHTHLM/document/20230821/3...,55907319
3,None,550001774480,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/nlp/cdp-xml/...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/nlp/ner/2023...,430,2023-08-21 04:00:00,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/nlp/pcs-xml/...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/preprocessin...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/preprocessin...,None,...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/eandmcs/2023...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/ruleengine/2...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/cptcs/202308...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/cptcs/202308...,10943184,9270967,R00.2,12096201,SJRMC/data/CM_ADLT_OPHTHLM/document/20230821/3...,55907319
4,None,550001774480,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/nlp/cdp-xml/...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/nlp/ner/2023...,430,2023-08-21 04:00:00,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/nlp/pcs-xml/...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/preprocessin...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/preprocessin...,None,...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/eandmcs/2023...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/ruleengine/2...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/cptcs/202308...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/cptcs/202308...,10943184,9270967,R06.02,12096201,SJRMC/data/CM_ADLT_OPHTHLM/document/20230821/3...,55907319
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
729,None,550001747411,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/nlp/cdp-xml/...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/nlp/ner/2023...,430,2023-07-26 04:00:00,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/nlp/pcs-xml/...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/preprocessin...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/preprocessin...,None,...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/eandmcs/2023...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/ruleengine/2...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/cptcs/202307...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/cptcs/202307...,10213972,8693803,H02.89,11215059,SJRMC/data/CM_ADLT_OPHTHLM/docum

In [55]:
# df_query.to_excel('/home/trishalaswain/Documents/AGS_PROJECTS/OCR_SERVICE/SCP_3665_PCS/3665_Output.xlsx')

In [56]:
df_query.account_number=df_query.account_number.apply(lambda x:str(x))

In [57]:
df_query

,ocr_request_id,account_number,nlp_request_s3_path,nlp_ner_entities_result_s3_path,facility_id,service_date,nlp_pcs_xml_result_s3_path,preprocessing_request_s3_path,preprocessing_result_s3_path,orignal_path,...,eandmcs_result_s3_path,ruleengine_request_s3_path,cptcs_request_s3_path,cptcs_result_s3_path,account_id,encounter_id,suggested_code,DPD_ID,filePath,document_id
0,None,550001774480,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/nlp/cdp-xml/...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/nlp/ner/2023...,430,2023-08-21 04:00:00,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/nlp/pcs-xml/...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/preprocessin...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/preprocessin...,None,...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/eandmcs/2023...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/ruleengine/2...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/cptcs/202308...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/cptcs/202308...,10943184,9270967,E11.9,12096201,SJRMC/data/CM_ADLT_OPHTHLM/document/20230821/3...,55907319
1,None,550001774480,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/nlp/cdp-xml/...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/nlp/ner/2023...,430,2023-08-21 04:00:00,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/nlp/pcs-xml/...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/preprocessin...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/preprocessin...,None,...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/eandmcs/2023...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/ruleengine/2...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/cptcs/202308...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/cptcs/202308...,10943184,9270967,M32.9,12096201,SJRMC/data/CM_ADLT_OPHTHLM/document/20230821/3...,55907319
2,None,550001774480,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/nlp/cdp-xml/...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/nlp/ner/2023...,430,2023-08-21 04:00:00,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/nlp/pcs-xml/...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/preprocessin...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/preprocessin...,None,...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/eandmcs/2023...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/ruleengine/2...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/cptcs/202308...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/cptcs/202308...,10943184,9270967,90853,12096201,SJRMC/data/CM_ADLT_OPHTHLM/document/20230821/3...,55907319
3,None,550001774480,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/nlp/cdp-xml/...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/nlp/ner/2023...,430,2023-08-21 04:00:00,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/nlp/pcs-xml/...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/preprocessin...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/preprocessin...,None,...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/eandmcs/2023...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/ruleengine/2...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/cptcs/202308...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/cptcs/202308...,10943184,9270967,R00.2,12096201,SJRMC/data/CM_ADLT_OPHTHLM/document/20230821/3...,55907319
4,None,550001774480,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/nlp/cdp-xml/...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/nlp/ner/2023...,430,2023-08-21 04:00:00,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/nlp/pcs-xml/...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/preprocessin...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/preprocessin...,None,...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/eandmcs/2023...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/ruleengine/2...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/cptcs/202308...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/cptcs/202308...,10943184,9270967,R06.02,12096201,SJRMC/data/CM_ADLT_OPHTHLM/document/20230821/3...,55907319
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
729,None,550001747411,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/nlp/cdp-xml/...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/nlp/ner/2023...,430,2023-07-26 04:00:00,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/nlp/pcs-xml/...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/preprocessin...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/preprocessin...,None,...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/eandmcs/2023...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/ruleengine/2...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/cptcs/202307...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/cptcs/202307...,10213972,8693803,H02.89,11215059,SJRMC/data/CM_ADLT_OPHTHLM/docum

In [58]:
df_query.groupby(["encounter_id"]).size()

encounter_id
8693803    12
8694302    14
8695490    10
8695860    19
8697044    13
8698099    11
8698420    10
8698878    13
8699408    14
8798797     9
8799326    14
8800601     7
8801170    15
8801719    13
8802608    14
8803732    17
8804292    11
8804649    13
8846046    14
8846088    14
8846102    13
8846202    11
8846256    13
8846415    16
8856672    14
8857447    14
8857977    18
8858916    12
8859366    13
8860725    16
8861322    11
9121444    15
9122088    16
9122574     2
9123536    14
9124190     5
9124665    10
9125372    14
9126064     6
9126874    10
9139520     1
9139558    12
9139765    15
9139830    11
9139896    17
9140021    13
9140077    15
9140108     8
9140175    10
9140365     8
9183420    10
9183793    16
9184380    13
9184845    14
9185581    12
9269957    15
9270035    13
9270468    15
9270967    16
dtype: int64

In [59]:
df_query=df_query.groupby(["filePath", "service_date", 'facility_id', 'type_id', 'dpd_id']).first().reset_index()

In [60]:
df_query=df_query.sort_values(['facility_id', 'type_id'])

In [61]:
df_query.columns

Index(['filePath', 'service_date', 'facility_id', 'type_id', 'dpd_id',
       'ocr_request_id', 'account_number', 'nlp_request_s3_path',
       'nlp_ner_entities_result_s3_path', 'nlp_pcs_xml_result_s3_path',
       'preprocessing_request_s3_path', 'preprocessing_result_s3_path',
       'orignal_path', 'created_date', 'ocr_request_s3_path',
       'ocr_result_s3_path', 'eandmcs_request_s3_path',
       'eandmcs_result_s3_path', 'ruleengine_request_s3_path',
       'cptcs_request_s3_path', 'cptcs_result_s3_path', 'account_id',
       'encounter_id', 'suggested_code', 'DPD_ID', 'document_id'],
      dtype='object')

In [62]:
len(df_query)

59

In [63]:
df_query.DPD_ID.value_counts()

11215059    1
11215734    1
11218204    1
11220653    1
11219034    1
11222139    1
11223357    1
11222704    1
11566337    1
11359354    1
11373253    1
11361795    1
11362557    1
11363335    1
11364479    1
11365763    1
11366495    1
11373242    1
11420343    1
11432869    1
11420422    1
11433785    1
11420452    1
11445701    1
11420619    1
11435625    1
11420737    1
11436245    1
11420937    1
11437907    1
11438565    1
11806384    1
11831408    1
11808984    1
11831567    1
11809739    1
11831793    1
11810858    1
11831757    1
11811780    1
11831175    1
11812382    1
11831698    1
11813233    1
11831492    1
11813956    1
11832138    1
11815501    1
11831866    1
11831129    1
11916855    1
11917439    1
11918193    1
11951607    1
11919924    1
12055004    1
12055806    1
12055111    1
12096201    1
Name: DPD_ID, dtype: int64

In [64]:
df_query.account_number.value_counts()

550001757541    2
550001757196    2
550001763465    2
550001754826    2
550001754791    2
550001755325    2
550001767934    2
550001754846    2
550001774480    2
550001771905    2
550001767547    2
550001771465    2
550001767751    2
550001767681    2
550001754790    2
550001774527    2
550001771886    2
550001767935    2
550001777290    2
550001747854    1
550001747411    1
550001747743    1
550001747381    1
550001748269    1
550001762968    1
550001765943    1
550001753280    1
550001757236    1
550001753142    1
550001753113    1
550001753144    1
550001753284    1
550001753141    1
550001753228    1
550001767620    1
550001769502    1
550001767167    1
550001769462    1
550001769640    1
550001769625    1
Name: account_number, dtype: int64

In [65]:
df_query[df_query['account_number']=='90284753']

,filePath,service_date,facility_id,type_id,dpd_id,ocr_request_id,account_number,nlp_request_s3_path,nlp_ner_entities_result_s3_path,nlp_pcs_xml_result_s3_path,...,eandmcs_request_s3_path,eandmcs_result_s3_path,ruleengine_request_s3_path,cptcs_request_s3_path,cptcs_result_s3_path,account_id,encounter_id,suggested_code,DPD_ID,document_id


In [66]:
# df_query.to_excel('/home/trishalaswain/ClientData/SCP/df_query.xlsx')

In [67]:
print(df_query.groupby('facility_id').size().to_markdown())

|   facility_id |   0 |
|--------------:|----:|
|           430 |  59 |


In [68]:
# df_query.to_excel('/home/trishalaswain/Downloads/docs_4525_MAY_JUN_JUL_DATA (1)/df_query.xlsx')

In [84]:
df_query.account_number.value_counts()

550001757541    2
550001757196    2
550001763465    2
550001754826    2
550001754791    2
550001755325    2
550001767934    2
550001754846    2
550001774480    2
550001771905    2
550001767547    2
550001771465    2
550001767751    2
550001767681    2
550001754790    2
550001774527    2
550001771886    2
550001767935    2
550001777290    2
550001747854    1
550001747411    1
550001747743    1
550001747381    1
550001748269    1
550001762968    1
550001765943    1
550001753280    1
550001757236    1
550001753142    1
550001753113    1
550001753144    1
550001753284    1
550001753141    1
550001753228    1
550001767620    1
550001769502    1
550001767167    1
550001769462    1
550001769640    1
550001769625    1
Name: account_number, dtype: int64

In [86]:
df_query[df_query.account_number == '550001757196']

,filePath,service_date,facility_id,type_id,dpd_id,ocr_request_id,account_number,nlp_request_s3_path,nlp_ner_entities_result_s3_path,nlp_pcs_xml_result_s3_path,...,eandmcs_request_s3_path,eandmcs_result_s3_path,ruleengine_request_s3_path,cptcs_request_s3_path,cptcs_result_s3_path,account_id,encounter_id,suggested_code,DPD_ID,document_id
5,SJRMC/data/CM_ADLT_OPHTHLM/document/20230726/2...,2023-07-26 04:00:00,430,1647013,11222139,None,550001757196,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/nlp/cdp-xml/...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/nlp/ner/2023...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/nlp/pcs-xml/...,...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/eandmcs/2023...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/eandmcs/2023...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/ruleengine/2...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/cptcs/202307...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/cptcs/202307...,10459664,8698099,R06.02,11222139,52381480
6,SJRMC/data/CM_ADLT_OPHTHLM/document/20230726/2...,2023-07-26 04:00:00,430,1647013,11223357,None,550001757196,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/nlp/cdp-xml/...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/nlp/ner/2023...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/nlp/pcs-xml/...,...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/eandmcs/2023...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/eandmcs/2023...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/ruleengine/2...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/cptcs/202307...,ezcapc/SJRMC/data/CM_ADLT_OPHTHLM/cptcs/202307...,10459664,8698878,Q38.8,11223357,52385487


In [75]:
def parse_and_create_script(x):
    print(x.facility_id.unique())
    
    df_query=x
    cmd=[]
    fac_list=[]

    for dpd_id, document_id, pdf,original_text,pcs,em_req,em_res,acc,ori_path, pre_req, pre_res, acc_id,ti, nlp_req, nlp_res, cpt_req, ctp_res, rule_req, enc_id,suggested_code, dpd_id,t_id, file_path in zip(df_query.dpd_id, df_query.document_id, df_query.ocr_request_s3_path,df_query.ocr_result_s3_path,
                                                                                          df_query.nlp_pcs_xml_result_s3_path,
                                                             df_query.eandmcs_request_s3_path,df_query.eandmcs_result_s3_path, df_query.account_number, df_query.orignal_path,
                                                             df_query.preprocessing_request_s3_path, df_query.preprocessing_result_s3_path,
                                                             df_query.account_id, df_query.type_id, 
                                                             df_query.nlp_request_s3_path, df_query.nlp_ner_entities_result_s3_path, 
                                                             df_query.cptcs_request_s3_path, df_query.cptcs_result_s3_path,
                                                             df_query.ruleengine_request_s3_path,
                                                             df_query.encounter_id,
                                                             df_query.suggested_code, 
                                                            #  df_query.final_code,
                                                             df_query.DPD_ID,
                                                             df_query.type_id,
                                                             df_query.filePath
                                                            ):
        if pcs == None or acc==None or em_req==None:
            continue
        fac_name=file_path.split("/")[2]
        cli=file_path.split("/")[0]
        fac_list.append(fac_name)

        # eandm
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}_{}_{}_pcs.xml".format(env, pcs, fac_name, fac_name, enc_id, acc))
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}_{}_{}_eandm_request.json".format(env, em_req, fac_name, fac_name, enc_id, acc))
        # cmd.append((pcs, '{}/{}_{}_{}_pcs.xml'.format(fac_name, fac_name, enc_id, acc)))
        # cmd.append((pcs, '{}/{}/{}_{}_{}_{}_pcs.xml'.format(fac_name, enc_id, enc_id, acc, suggested_code, final_code)))
        cmd.append((pcs, '{}/{}/{}_{}_pcs.xml'.format(fac_name, dpd_id, enc_id, acc)))

        # cmd.append((em_req, '{}/{}_{}_{}_eandm_req.json'.format(fac_name, fac_name, enc_id, acc)))
        # cmd.append((em_req, '{}/{}/{}_{}_{}_eandm_req.json'.format(fac_name, enc_id, enc_id, acc, suggested_code)))

        
        # analysis data
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_{}_{}_{}_{}_{}_pcs.xml".format(env, pcs, fac_name, enc_id, fac_name, enc_id, acc, document_id, suggested_code, final_code))
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_{}_{}_{}_{}_{}_eandm_request.json".format(env, em_req, fac_name, enc_id, fac_name, enc_id, acc, document_id, suggested_code, final_code))
        # cmd.append((pcs, '{}/{}/{}_{}_{}_{}_{}_{}_pcs.xml'.format(fac_name, enc_id, fac_name, enc_id, acc, document_id, suggested_code, final_code)))
        # cmd.append((em_req, '{}/{}/{}_{}_{}_{}_{}_{}_eandm_req.json'.format(fac_name, enc_id, fac_name, enc_id, acc, document_id, suggested_code, final_code)))
        
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_result.json".format(env, em_res,fac_name, acc_id, acc))
        # cmd.append((em_res, '{}/{}/{}_{}_eandm_res.json'.format(fac_name,dpd_id, acc, t_id)))

        # cpt
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_{}_pcs.xml".format(env, pcs, fac_name, enc_id, dpd_id, t_id))
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_{}_{}_cpt_request.json".format(env, cpt_req, fac_name, enc_id,fac_name, enc_id, acc))

        # cmd.append((pcs, '{}/{}/{}_{}_pcs.xml'.format(fac_name, dpd_id, enc_id, acc)))
        cmd.append((cpt_req, '{}/{}/{}_{}_cpt_req.json'.format(fac_name, dpd_id, enc_id, acc)))
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_result.json".format(env, ctp_res,fac_name, acc_id, acc))

        # preprocess
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_preprocess_request.txt".format(env, pre_req, fac_name, enc_id,acc))
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_preprocess_result.txt".format(env, pre_res, fac_name,enc_id, acc))

        # original path
        # print("aws s3 cp 's3://ezdi-{}-bucket/ezcapc/{}' {}/{}/{}_{}_{}_{}.pdf".format(env,ori_path,fac_name, enc_id, t_id, acc, suggested_code, final_code))
        # cmd.append((ori_path, '{}/{}/{}_{}_{}_{}.pdf'.format(fac_name, enc_id, t_id, acc, suggested_code, final_code)))
        # print("aws s3 cp 's3://ezdi-{}-bucket/ezcapc/{}' {}/{}/{}.txt".format(env,file_path,fac_name,t_id,acc))

        # ocr
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}.txt".format(env,original_text,fac_name,acc_id,acc))
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}.pdf".format(env,pdf,fac_name,acc_id,acc))

        # nlp
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}_{}_{}_NLPrequest.json".format(env, nlp_req, fac_name,enc_id, acc))
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}_{}_{}_NLPresult.json".format(env, nlp_res, fac_name,enc_id, acc))
        # cmd.append((nlp_req, '{}/{}/{}_{}_{}_{}_nlp_req.json'.format(fac_name,enc_id,enc_id, acc, suggested_code, final_code)))
        # cmd.append((nlp_res, '{}/{}/{}_{}_{}_{}_nlp_res.json'.format(fac_name,enc_id,enc_id, acc, suggested_code, final_code)))

        # rule
        # print("aws s3 cp 's3://ezdi-{}-bucket/{}' {}/{}/{}_rule_req.json".format(env, rule_req,fac_name,acc_id, acc))

        # print(pre)

    cmd.sort()
    commands=[]

    for c in cmd:
        temp=c[0].rsplit('/', 1)
        # print(temp)
        if len(commands) == 0:
            commands.append(temp[0] + ' . --recursive --exclude "*" --include "' + temp[1] +'"')
        elif temp[0] in commands[-1]:
            if 'exclude' in commands[-1]:
                commands[-1] = commands[-1] + ' --include "' + temp[1] + '"'
        else:
            commands.append(temp[0] + ' . --recursive --exclude "*" --include "' + temp[1] + '"')
        # print('-----------------------------------------')
        # print(temp[0] + ' . --recursive --exclude "*" --include "' + temp[1] + '"')
    return cmd, commands, fac_name

In [76]:
def print_downlaod_files(commands,cmd, fac):
    print(len(commands))
    time_now=datetime.now()
    time_now='{}-{}-{}-{}-{}-{}'.format(time_now.day,time_now.month,time_now.hour,time_now.minute,time_now.second,time_now.microsecond)

    file=open("/home/trishalaswain/Documents/AGS_PROJECTS/{}_docs_download_{}.sh".format(fac,time_now),'w')
    for f in commands:
        file.write('aws s3 cp s3://ezdi-{}-bucket/'.format(env) + re.sub('_\w+.\d{3}_', '*',f) +'\n')
        # print('===================')
        # print(f)
    # file.write('sh {}_name_change_*'.format(fac))
    
    for f in cmd:
        # print(f)
        f=(f[0].rsplit('/', 1), f[1])
        # print('mv "'+f[0][1] +'" ' +f[1])
        file.write('mkdir --parents ./' + f[1].rsplit('/',1)[0] + '/; mv "' + f[0][1] + '" '+f[1]+'\n')
    file.write('tar -cJf docs_{}.tar.xz {}\n'.format(fac, fac))
    file.write('rm -r {}'.format(fac))
    file.close()


In [77]:
# def print_filename_change(cmd, fac):
#     time_now=datetime.now()
#     time_now='{}-{}-{}-{}-{}-{}'.format(time_now.day,time_now.month,time_now.hour,time_now.minute,time_now.second,time_now.microsecond)

#     file=open("/home/devanshparmar/devansh/Download/{}_name_change_{}.sh".format(fac,time_now),'w')
#     for f in cmd:
#         # print(f)
#         f=(f[0].rsplit('/', 1), f[1])
#         # print('mv "'+f[0][1] +'" ' +f[1])
#         file.write('mkdir --parents ./' + f[1].rsplit('/',1)[0] + '/; mv "' + f[0][1] + '" '+f[1]+'\n')
#     file.write('tar -cJf docs_{}.tar.xz {}\n'.format(fac, fac))
#     file.write('rm -r {}'.format(fac))
#     file.close()

In [78]:
def do_all(x):
    # print(len(commands))
    # for cmd in commands:
    # print(commands[0], commands[-1])
    
    cmd, commands, fac = parse_and_create_script(x)
    print_downlaod_files(commands, cmd, fac)
    # print_filename_change(cmd, fac)


In [79]:
df_query.groupby('facility_id').apply(lambda x : do_all(x))

[430]
12


""


In [ ]:
df_query.columns

In [ ]:
# df_query[df_query.account_number.str.contains('0996')]

In [ ]:
# f=open("/home/devanshparmar/devansh/Download/docs_chs.sh".format(cli,fac_name),'a')
# for d in cmd:
#     f.write(d+'\n')
    
# f.write('tar -cJf docs.tar.xz *\n')
# f.write('echo done\n')
# f.close()

In [ ]:
# import os
# root = '/home/trishalaswain/Downloads/docs_4525_MAY_JUN_JUL_DATA/4525_Time_Docs/'
# for file in os.listdir(root):
#     if '99291_992' not in file:
#         print(file)
#         path = os.path.join(root + '/' + file)
#         os.remove(path)

In [ ]:

# for f_id in type_id:
# query ='''
# SELECT DISTINCT
#     am.account_number,
#     dpd.nlp_request_s3_path,
#     dpd.nlp_ner_entities_result_s3_path,
#     am.facility_id,
#     dm.service_date,
#     dpd.nlp_pcs_xml_result_s3_path,
#     dpd.preprocessing_request_s3_path,
#     dpd.preprocessing_result_s3_path,
#     dm.orignal_path,
#     dm.type_id,
#     dpd.created_date,
#     dpd.ocr_request_s3_path,
#     dpd.ocr_result_s3_path,
#     dpd.eandmcs_request_s3_path,
#     dpd.eandmcs_result_s3_path,
#     dpd.ruleengine_request_s3_path,
#     dpd.cptcs_request_s3_path,
#     dpd.cptcs_result_s3_path,
#     dm.account_id,
#     dpd.id AS DPD_ID,
#     dm.path AS filePath,
#     dm.id as document_id,
#     em.id as encounter_id
# FROM
#     account_mst am
#         INNER JOIN
#     document_mst dm ON dm.account_id = am.id
#         INNER JOIN
#     document_processing_detail dpd ON dpd.document_id = dm.id
#         INNER JOIN
#     encounter_mst em ON dm.account_id = em.account_id
#         AND dm.service_date = em.service_date
#         AND dm.physician_id = em.physician_id
#         AND em.is_active = 1
# WHERE
#         is_physician_document = 1
#         AND dm.is_active = 1
#         and account_number in ({})
#         and dpd.id = (select MAX(id)FROM document_processing_detail
#           WHERE document_id = dm.id)

# ORDER BY dm.created_date DESC;
# '''.format(','.join(['"'+str(x)+'"' for x in acc]))
# print(query)

# for f in fac:

# query='''
# SELECT DISTINCT
#     am.account_number,
#     dpd.nlp_request_s3_path,
#     dpd.nlp_ner_entities_result_s3_path,
#     am.facility_id,
#     dm.service_date,
#     dpd.nlp_pcs_xml_result_s3_path,
#     dpd.preprocessing_request_s3_path,
#     dpd.preprocessing_result_s3_path,
#     dm.orignal_path,
#     dm.type_id,
#     dpd.created_date,
#     dpd.ocr_request_s3_path,
#     dpd.ocr_result_s3_path,
#     dpd.eandmcs_request_s3_path,
#     dpd.eandmcs_result_s3_path,
#     dpd.ruleengine_request_s3_path,
#     dpd.cptcs_request_s3_path,
#     dpd.cptcs_result_s3_path,
#     dm.account_id,
#     em.id as encounter_id,
#     dc.code AS suggested_code,
#     dpd.id AS DPD_ID,
#     dm.path as filePath,
#     ec.code AS final_code
# FROM
#     account_mst am
#         JOIN
#     document_mst dm ON dm.account_id = am.id
#         JOIN
#     document_processing_detail dpd ON dpd.document_id = dm.id
#         INNER JOIN
#     encounter_mst em ON am.id = em.account_id
#         left JOIN
#     document_code dc ON dc.document_id = dm.id
#         left JOIN
#     eandm_code ec ON ec.encounter_id = em.id
# WHERE
#     dc.code_category = 'EANDM'
#         and is_physician_document=1
#         and dm.is_active=1
#         and account_number in ({})
#         and dpd.id = (SELECT MAX(id)
#         FROM document_processing_detail
#         WHERE document_id = dm.id)
#     order by dm.created_date desc;
# '''.format(','.join(str(x) for x in acc))

# query = '''SELECT DISTINCT
#     am.account_number,
#     dpd.nlp_request_s3_path,
#     dpd.nlp_ner_entities_result_s3_path,
#     am.facility_id,
#     dm.service_date,
#     dpd.nlp_pcs_xml_result_s3_path,
#     dpd.preprocessing_request_s3_path,
#     dpd.preprocessing_result_s3_path,
#     dm.orignal_path,
#     dm.type_id,
#     dpd.created_date,
#     dpd.ocr_request_s3_path,
#     dpd.ocr_result_s3_path,
#     dpd.eandmcs_request_s3_path,
#     dpd.eandmcs_result_s3_path,
#     dpd.ruleengine_request_s3_path,
#     dpd.cptcs_request_s3_path,
#     dpd.cptcs_result_s3_path,
#     dm.account_id,
#     em.id AS encounter_id,
#     dc.code AS suggested_code,
#     dpd.id AS DPD_ID,
#     dm.path AS filePath,
#     ec.code AS final_code
# FROM
#     account_mst am
#         JOIN
#     document_mst dm ON dm.account_id = am.id
#         JOIN
#     document_processing_detail dpd ON dpd.document_id = dm.id
#         INNER JOIN
#     encounter_mst em ON am.id = em.account_id
#         LEFT JOIN
#     document_code dc ON dc.document_id = dm.id
#         LEFT JOIN
#     eandm_code ec ON ec.encounter_id = em.id
# WHERE
#         dc.code_category = 'EANDM'
#         is_physician_document = 1
#         AND am.account_number in ({})
#         AND dm.is_active = 1
#         AND am.is_active = 1
# ORDER BY dpd.created_date DESC;'''.format(','.join(str(x) for x in acc))

           
# query = '''SELECT DISTINCT
#     am.account_number,
#     dpd.nlp_request_s3_path,
#     dpd.nlp_ner_entities_result_s3_path,
#     am.facility_id,
#     dm.service_date,
#     dpd.nlp_pcs_xml_result_s3_path,
#     dpd.preprocessing_request_s3_path,
#     dpd.preprocessing_result_s3_path,
#     dm.orignal_path,
#     dm.type_id,
#     dpd.created_date,
#     dpd.ocr_request_s3_path,
#     dpd.ocr_result_s3_path,
#     dpd.eandmcs_request_s3_path,
#     dpd.eandmcs_result_s3_path,
#     dpd.ruleengine_request_s3_path,
#     dpd.cptcs_request_s3_path,
#     dpd.cptcs_result_s3_path,
#     dm.account_id,
#     em.id AS encounter_id,
#     dc.code AS suggested_code,
#     dpd.id AS DPD_ID,
#     dm.path AS filePath,
#     ec.code AS final_code
# FROM
#     account_mst am
#         JOIN
#     document_mst dm ON dm.account_id = am.id
#         JOIN
#     document_processing_detail dpd ON dpd.document_id = dm.id
#         INNER JOIN
#     encounter_mst em ON am.id = em.account_id
#         LEFT JOIN
#     document_code dc ON dc.document_id = dm.id
#         LEFT JOIN
#     eandm_code ec ON ec.encounter_id = em.id
# WHERE
#     am.facility_id = 38
#         AND is_physician_document = 1
#         AND dm.service_date >= '2023-05-01'
#         AND dm.is_active = 1
#         AND am.is_active = 1
# ORDER BY dpd.created_date DESC
# '''

In [ ]:
# query='''
# SELECT DISTINCT
#     am.account_number,
#     dpd.nlp_request_s3_path,
#     dpd.nlp_ner_entities_result_s3_path,
#     am.facility_id,
#     dm.service_date,
#     dpd.nlp_pcs_xml_result_s3_path,
#     dpd.preprocessing_request_s3_path,
#     dpd.preprocessing_result_s3_path,
#     dm.orignal_path,
#     dm.type_id,
#     dpd.created_date,
#     dpd.ocr_request_s3_path,
#     dpd.ocr_result_s3_path,
#     dpd.eandmcs_request_s3_path,
#     dpd.eandmcs_result_s3_path,
#     dpd.ruleengine_request_s3_path,
#     dpd.cptcs_request_s3_path,
#     dpd.cptcs_result_s3_path,
#     dm.account_id,
#     em.id as encounter_id,
#     dc.code AS suggested_code,
#     dpd.id AS DPD_ID,
#     dm.path as filePath,
#     ec.code AS final_code
# FROM
#     account_mst am
#         JOIN
#     document_mst dm ON dm.account_id = am.id
#         JOIN
#     document_processing_detail dpd ON dpd.document_id = dm.id
#         INNER JOIN
#     encounter_mst em ON am.id = em.account_id
#         INNER JOIN
#     document_code dc ON dc.document_id = dm.id
#         INNER JOIN
#     eandm_code ec ON ec.encounter_id = em.id
#         INNER JOIN
#     encounter_status_map esm ON esm.encounter_id = em.id
#         AND esm.is_active = 1
#         AND esm.encounter_status_id IN (9 , 17)
# WHERE
#         dc.code_category = 'EANDM'
#         and is_physician_document=1
#         and dm.is_active=1
#         and em.id in ({})
#         and dpd.id = (SELECT MAX(id)
#         FROM document_processing_detail
#         WHERE document_id = dm.id)
#         and CONVERT_TZ(esm.last_coding_date,
#             'UTC',
#             'US/Central') >= '2023-07-01'
#     order by dm.created_date desc;
# '''.format(','.join(['"'+str(x)+'"' for x in acc]))



# query = '''SELECT 
#     am.account_number,
#     dpd.nlp_request_s3_path,
#     dpd.nlp_ner_entities_result_s3_path,
#     am.facility_id,
#     dm.service_date,
#     dpd.nlp_pcs_xml_result_s3_path,
#     dpd.preprocessing_request_s3_path,
#     dpd.preprocessing_result_s3_path,
#     dm.orignal_path,
#     dm.type_id,
#     dpd.created_date,
#     dpd.ocr_request_s3_path,
#     dpd.ocr_result_s3_path,
#     dpd.eandmcs_request_s3_path,
#     dpd.eandmcs_result_s3_path,
#     dpd.ruleengine_request_s3_path,
#     dpd.cptcs_request_s3_path,
#     dpd.cptcs_result_s3_path,
#     dm.account_id,
#     em.id as encounter_id,
#     dc.code AS suggested_code,
#     dpd.id AS DPD_ID,
#     dm.path as filePath,
#     eandm.code AS final_code
# FROM
#     account_mst am
#         INNER JOIN
#     document_mst dm ON dm.account_id = am.id
#         AND dm.is_physician_document = 1
#         AND dm.is_active = 1
#         INNER JOIN
#     document_processing_detail dpd ON dpd.document_id = dm.id
#         INNER JOIN
#     document_code dc ON dc.document_id = dm.id
#         AND dc.is_active = 1
#         AND dc.code_category = 'EANDM'
#         INNER JOIN
#     encounter_mst em ON em.account_id = dm.account_id
#         AND em.service_date = dm.service_date
#         AND em.physician_id = dm.physician_id
#         AND em.is_active = 1
#         INNER JOIN
#     encounter_status_map esm ON esm.encounter_id = em.id
#         AND esm.is_active = 1
#         AND esm.encounter_status_id IN (9 , 17)
#         INNER JOIN
#     eandm_code eandm ON eandm.encounter_id = em.id
#         AND eandm.process_id = esm.process_id
#         AND eandm.user_id = esm.user_id
#         AND eandm.is_active = 1
	
# WHERE
# dpd.id = (SELECT 
#             id
#         FROM
#             document_processing_detail
#         WHERE
#             document_id = dm.id
#         ORDER BY created_date DESC
#         LIMIT 1)
#         AND am.facility_id = '81'
#         AND am.client_id = '15'
#         AND esm.id = (SELECT 
#             id
#         FROM
#             encounter_status_map
#         WHERE
#             encounter_id = em.id
#                 AND encounter_status_id IN (9 , 17)
#         ORDER BY last_coding_date DESC
#         LIMIT 1)
#         AND am.is_active = 1
#         AND CONVERT_TZ(esm.last_coding_date,
#             'UTC',
#             'US/Central') >= '2023-07-01'
#         AND CONVERT_TZ(em.service_date,
#             'UTC',
#             'US/Central') > '2023-07-01'
# GROUP BY am.account_number , em.physician_id , em.service_date , dm.id'''

    # facility_id,
    # client_id
    
    # CONVERT_TZ(em.service_date, 'UTC', 'US/Central') AS service_date,
    # GROUP_CONCAT(DISTINCT eandm.code) AS final,
    # GROUP_CONCAT(DISTINCT dc.code) AS suggested,
    # am.account_number AS account_number,
    
    
# query = '''SELECT DISTINCT
#     am.account_number,
#     dpd.nlp_request_s3_path,
#     dpd.nlp_ner_entities_result_s3_path,
#     am.facility_id,
#     dm.service_date,
#     dpd.nlp_pcs_xml_result_s3_path,
#     dpd.preprocessing_request_s3_path,
#     dpd.preprocessing_result_s3_path,
#     dm.orignal_path,
#     dm.type_id,
#     dpd.created_date,
#     dpd.ocr_request_s3_path,
#     dpd.ocr_result_s3_path,
#     dpd.eandmcs_request_s3_path,
#     dpd.eandmcs_result_s3_path,
#     dpd.ruleengine_request_s3_path,
#     dpd.cptcs_request_s3_path,
#     dpd.cptcs_result_s3_path,
#     dm.account_id,
#     em.id as encounter_id,
#     dc.code AS suggested_code,
#     dpd.id AS DPD_ID,
#     dm.path as filePath,
#     ec.code AS final_code
# FROM
#     account_mst am
#         JOIN
#     document_mst dm ON dm.account_id = am.id
#         JOIN
#     document_processing_detail dpd ON dpd.document_id = dm.id
#         INNER JOIN
#     encounter_mst em ON am.id = em.account_id
#         left JOIN
#     document_code dc ON dc.document_id = dm.id
#         left JOIN
#     eandm_code ec ON ec.encounter_id = em.id
# WHERE
#         is_physician_document=1
#         and dc.code_category = "EANDM"
#         and dpd.created_date >= '2023-07-16'
#         and am.facility_id = 264
#         and dm.is_active=1
#         and am.is_active=1'''